## Purpose
Annotates the pricing xlsx sheet with the inventory from wilco report.

It allows us to comput the value of 2 questions.
* What is the total dollar amount of WILCO inventory?
* What items are present in pricing sheet but not in wilco inventory? These can be manually checked to ensure that they exist at homte.

The final output for dollary amount is  *master_wilco_dollars.xlsx*

The various inputs are
* wilco inventory xlsx
* pricing xlsx to create a master list.
* lb inventory category

In [1]:
IN_DIR = "../private"
LB_CATEGORIES = "lb_inventory_categories.xlsx"


OUT_DIR = "out"
PRICING_XLSX = 'lb_sku_pricing.xlsx'
INVENTORY_XLSX = 'lb_sku_inventory.xlsx'

OUT_MASTER = "lb_inventory_master.xlsx"

In [6]:
import re
import os
import openpyxl
from pathlib import Path

import sys
sys.path.append("../src")
import lb_wilco_parser as lb


CATEGORIES_XLSX = Path(IN_DIR, LB_CATEGORIES)
IN_PRICING_XLSX = Path(OUT_DIR, PRICING_XLSX)
IN_INVENTORY_XLSX = Path(OUT_DIR, INVENTORY_XLSX)

OUT_XLSX = Path(OUT_DIR, OUT_MASTER)

if not os.path.exists(CATEGORIES_XLSX):
    raise ValueError("Categories not found")

if not os.path.exists(IN_PRICING_XLSX):
    raise ValueError("Pricing xlsx not found. Did you run the intermediate step?")

if not os.path.exists(IN_INVENTORY_XLSX):
    raise ValueError("Inventory xlsx not found. Did you run the intermediate step?")


In [7]:
wb = openpyxl.load_workbook(CATEGORIES_XLSX)
sheet = wb.active
categories = lb.create_categories_database(sheet)

Found 88 matching categories


In [8]:
print(categories)

{'001': ('Frozen', 'Parotta'), '003-20pk': ('Frozen', 'Tapioca'), '003-25pk': ('Frozen', 'Tapioca'), '004': ('Frozen', 'Coconut'), '010': ('Frozen', 'Coconut'), '022': ('Dry', 'Pickle'), '023': ('Dry', 'Pickle'), '026': ('Dry', 'Pickle'), '027': ('Dry', 'Pickle'), '028': ('Dry', 'Pickle'), '029': ('Dry', 'Pickle'), '036': ('Dry', 'Pickle'), '037': ('Dry', 'Pickle'), '038': ('Dry', 'Pickle'), '039': ('Dry', 'Pickle'), '041': ('Dry', 'Masala'), '042': ('Dry', 'Masala'), '044': ('Dry', 'Masala'), '048': ('Dry', 'Pickle'), '049SKU#: 36': ('Dry', 'Pickle'), '050': ('Dry', 'Pickle'), '054': ('Dry', 'Pickle'), '066': ('Dry', 'Snacks'), '068': ('Dry', 'Snacks'), '074': ('Dry', 'Masala'), '075': ('Dry', 'Pickle'), '076': ('Dry', 'Pickle'), '080': ('Dry', 'Pickle'), '083': ('Dry', 'Pickle'), '084': ('Dry', 'Pickle'), '086': ('Dry', 'Masala'), '087': ('Dry', 'Masala'), '088': ('Dry', 'Masala'), '089': ('Dry', 'Masala'), '090': ('Dry', 'Masala'), '091': ('Dry', 'Masala'), '092': ('Dry', 'Masala'),

In [12]:
wb = openpyxl.load_workbook(IN_INVENTORY_XLSX)
sheet = wb.active

In [13]:
inventory = {}
for row in range(2, sheet.max_row + 1):
    sku = sheet['A' + str(row)].value
    on_hand = sheet['C' + str(row)].value
    if sku in inventory:
        raise ValueError(f"Duplicate SKU {sku} found")
    inventory[sku] = on_hand
    inventory[

In [9]:
wb = openpyxl.load_workbook(IN_PRICING_XLSX)
sheet = wb.active
pricing = {}
for row in range(2, sheet.max_row + 1):
    sku = sheet['A' + str(row)].value
    details = (sheet['C' + str(row)].value, sheet['D' + str(row)].value, sheet['E' + str(row)].value)
    pricing[sku] = details
print(pricing)

{'022': (400, 12, 1.72), '029': (400, 12, 1.79), '036': (400, 12, 1.93), '034': (400, 12, 1.93), '032': (400, 12, 3.1), '027': (400, 12, 1.76), '028': (400, 12, 1.76), '023': (400, 12, 1.72), '038': (400, 12, 2.33), '039': (400, 12, 2.33), '037': (300, 12, 3.1), '042': (400, 12, 2.44), '041': (400, 12, 2.56), '043': (400, 12, 2.56), '044': (400, 12, 2.54), '073': (400, 12, 2.54), '074': (400, 12, 7.03), '048': (1000, 12, 2.66), '075': (1000, 12, 2.94), '051': (1000, 12, 2.87), '050': (1000, 12, 3.54), '049': (1000, 12, 2.65), '076': (1000, 12, 3.17), '080': (1000, 12, 3.76), '081': (1000, 12, 5.38), '082': (1000, 12, 3.43), '084': (1000, 12, 3.17), '054': (1000, 12, 3.5), '057': (600, 12, 3.83), '086': (200, 30, 2.16), '087': (200, 30, 2.12), '088': (200, 30, 2.28), '089': (200, 30, 2.21), '197': (250, 25, 2.7), '198': (250, 25, 2.66), 199: (250, 25, 2.84), '200': (250, 25, 2.77), '090': (250, 30, 1.87), '091': (250, 30, 1.87), '092': (250, 30, 1.6), '093': (250, 30, 2.8), 201: (325, 2

In [11]:
from openpyxl.styles import numbers

fmt_acc = u'$#,##0.00'

wb = openpyxl.load_workbook(IN_INVENTORY_XLSX)
sheet = wb.active
category_values = {}

sheet['D1'].value = "Type"
sheet['E1'].value = "Category"
sheet['F1'].value = "wt_pc_in_gms"
sheet['G1'].value = "pcs_per_case"
sheet['H1'].value = "price_per_pc"
sheet['I1'].value = "price_per_case"
sheet['J1'].value = "total_value"

missing_sku_pricing = []
for row in range(2, sheet.max_row + 1):
    sku = sheet['A' + str(row)].value
    if not sku in pricing:
        print(f"Missing {sku} in pricing")
        missing_sku_pricing.append(sku)
        continue
    details = pricing[sku]
    sheet['D' + str(row)].value = categories[sku][0]
    sheet['E' + str(row)].value = categories[sku][1]
    sheet['F' + str(row)].value = details[0]
    sheet['G' + str(row)].value = details[1]
    sheet['H' + str(row)].value = details[2]
    sheet['H' + str(row)].number_format = fmt_acc
    sheet['I' + str(row)].value = f"=G{row} * H{row}"
    sheet['I' + str(row)].number_format = fmt_acc
    sheet['J' + str(row)].value = f"=C{row} * I{row}"
    sheet['J' + str(row)].number_format = fmt_acc
    
wb.save(OUT_XLSX)

Missing 003-20pk in pricing
Missing 003-25pk in pricing
Missing 026 in pricing
Missing 049SKU#: 36 in pricing
Missing 068 in pricing
Missing 083 in pricing
Missing 102 in pricing
Missing 103 in pricing
Missing 104 in pricing
Missing 108 in pricing
Missing 110 in pricing
Missing 111 in pricing
Missing 112 in pricing
Missing 113 in pricing
Missing 115 in pricing
Missing 116 in pricing
Missing 118 in pricing
Missing 124 in pricing
Missing 137 in pricing
Missing 137-A in pricing
Missing 169 in pricing
Missing 199 in pricing
Missing 201 in pricing
Missing 202 in pricing
Missing 203 in pricing
Missing 204 in pricing
Missing 205 in pricing
Missing 206 in pricing
Missing CP1SKU#: 165,174,171,172 in pricing
Missing CP2SKU#: 168,166,170,167 in pricing
Missing CP3SKU#: 175,173,176,169 in pricing


In [12]:
print(f"Found {len(missing_sku_pricing)} missing pricing info")

Found 31 missing pricing info


In [13]:
print(missing_sku_pricing)

['003-20pk', '003-25pk', '026', '049SKU#: 36', '068', '083', '102', '103', '104', '108', '110', '111', '112', '113', '115', '116', '118', '124', '137', '137-A', '169', '199', '201', '202', '203', '204', '205', '206', 'CP1SKU#: 165,174,171,172', 'CP2SKU#: 168,166,170,167', 'CP3SKU#: 175,173,176,169']
